In [5]:
import pandas as pd

# Membaca file Excel
file_path = 'Data_Panel/data_panel.xlsx'
df = pd.read_excel(file_path)
# Menampilkan data
print(df)


             Provinsi      IPM  Tahun       PDRB  Realisasi_Investasi  \
0                ACEH  71.9900   2020  166372.32               8241.1   
1      SUMATERA UTARA  71.7700   2020  811188.31              18189.5   
2      SUMATERA BARAT  72.3800   2020  241894.13               3106.2   
3                RIAU  72.7100   2020  727599.47              34117.8   
4               JAMBI  71.2900   2020  205081.99               3511.7   
..                ...      ...    ...        ...                  ...   
147  PAPUA BARAT DAYA  33.3300   2023   36104.47               1708.7   
148             PAPUA  15.5625   2023   81737.92               1174.1   
149     PAPUA SELATAN  15.5625   2023   31359.66                152.5   
150      PAPUA TENGAH  15.5625   2023  150371.08                458.8   
151  PAPUA PEGUNUNGAN  15.5625   2023   24433.99                121.8   

     Laju_pdrb     tpt  Gini_Ratio  
0        -0.37  6.5900      0.3210  
1        -1.07  6.9100      0.3150  
2        -1.

In [6]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from scipy.stats import f

# Set index untuk panel data: Provinsi dan Tahun
df.set_index(['Provinsi', 'Tahun'], inplace=True)

# Variabel dependen (Laju_pdrb)
y = df['Laju_pdrb']

# Variabel independen
X = df[['IPM', 'PDRB', 'Realisasi_Investasi', "tpt",'Gini_Ratio']]

# Tambahkan konstanta (intercept)
X = sm.add_constant(X)

# 1. Fit Pooled OLS Model (Model tanpa efek tetap)
pooled_model = sm.OLS(y, X).fit()
RSS_pooled = np.sum(pooled_model.resid**2)

# 2. Fit Fixed Effects Model (dengan efek tetap untuk Provinsi)
fixed_effects_model = PanelOLS.from_formula('Laju_pdrb ~ IPM + PDRB + Realisasi_Investasi + EntityEffects + tpt + Gini_Ratio', data=df).fit()
RSS_fe = np.sum(fixed_effects_model.resids**2)

# 3. Chow Test - Hitung Statistik F
n = len(df.index.get_level_values('Provinsi').unique())  # Jumlah entitas (provinsi)
k = X.shape[1]  
T = len(df.index.get_level_values('Tahun').unique())  # Jumlah periode waktu
NT = len(df)  # Total jumlah observasi

# Hitung F-statistik untuk Chow Test
F_stat = ((RSS_pooled - RSS_fe) / (n - k)) / (RSS_fe / (NT - n - k))

# Derajat kebebasan untuk Chow Test
df1 = n - k  # Derajat kebebasan untuk pembilang
df2 = NT - n - k  # Derajat kebebasan untuk penyebut

# Hitung p-value untuk F-statistik
p_value = 1 - f.cdf(F_stat, df1, df2)

# Output F-statistik dan p-value
print(f"Chow Test F-statistic: {F_stat}")
print(f"Chow Test p-value: {p_value}")

# Kesimpulan
if p_value < 0.05:
    print("Reject H0: Gunakan model Fixed Effects.")
else:
    print("Fail to reject H0: Gunakan model Pooled OLS.")


Chow Test F-statistic: 7.928595930990043
Chow Test p-value: 1.1102230246251565e-16
Reject H0: Gunakan model Fixed Effects.


In [7]:
import pandas as pd

# Membaca file Excel
file_path = 'Data_Panel/data_panel.xlsx'
df = pd.read_excel(file_path)
# Menampilkan data
print(df)


             Provinsi      IPM  Tahun       PDRB  Realisasi_Investasi  \
0                ACEH  71.9900   2020  166372.32               8241.1   
1      SUMATERA UTARA  71.7700   2020  811188.31              18189.5   
2      SUMATERA BARAT  72.3800   2020  241894.13               3106.2   
3                RIAU  72.7100   2020  727599.47              34117.8   
4               JAMBI  71.2900   2020  205081.99               3511.7   
..                ...      ...    ...        ...                  ...   
147  PAPUA BARAT DAYA  33.3300   2023   36104.47               1708.7   
148             PAPUA  15.5625   2023   81737.92               1174.1   
149     PAPUA SELATAN  15.5625   2023   31359.66                152.5   
150      PAPUA TENGAH  15.5625   2023  150371.08                458.8   
151  PAPUA PEGUNUNGAN  15.5625   2023   24433.99                121.8   

     Laju_pdrb     tpt  Gini_Ratio  
0        -0.37  6.5900      0.3210  
1        -1.07  6.9100      0.3150  
2        -1.

In [8]:
# Convert Tahun to numeric

df=df.set_index(['Provinsi', 'Tahun'])
from statsmodels.api import add_constant
# Variabel dependen (PDRB)
Y = df['Laju_pdrb']

# Variabel independen
X = add_constant(df[['IPM', 'PDRB', 'Realisasi_Investasi', "tpt","Gini_Ratio"]])

# Pooled OLS
model = PooledOLS(Y, X)
ce_results = model.fit()

# Output hasil
print(ce_results)

                          PooledOLS Estimation Summary                          
Dep. Variable:              Laju_pdrb   R-squared:                        0.1793
Estimator:                  PooledOLS   R-squared (Between):             -0.0222
No. Observations:                 152   R-squared (Within):               0.3414
Date:                Thu, Nov 28 2024   R-squared (Overall):              0.1793
Time:                        16:56:36   Log-likelihood                   -409.35
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      6.3791
Entities:                          38   P-value                           0.0000
Avg Obs:                       4.0000   Distribution:                   F(5,146)
Min Obs:                       4.0000                                           
Max Obs:                       4.0000   F-statistic (robust):             6.3791
                            

In [9]:
# Model Fixed Effects
model_fe = PanelOLS(Y, X, entity_effects=True)
fe_results = model_fe.fit()
# Menampilkan hasil regresi
print(fe_results)

                          PanelOLS Estimation Summary                           
Dep. Variable:              Laju_pdrb   R-squared:                        0.5579
Estimator:                   PanelOLS   R-squared (Between):             -13.344
No. Observations:                 152   R-squared (Within):               0.5579
Date:                Thu, Nov 28 2024   R-squared (Overall):             -5.6388
Time:                        16:56:36   Log-likelihood                   -317.49
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      27.507
Entities:                          38   P-value                           0.0000
Avg Obs:                       4.0000   Distribution:                   F(5,109)
Min Obs:                       4.0000                                           
Max Obs:                       4.0000   F-statistic (robust):             27.507
                            

In [10]:
#Penerapan Model Random Effects
# Model Random Effects
model_re = RandomEffects(Y, X)
re_results = model_re.fit()
#Menampilkan hasil regresi
print(re_results)
#Uji Hausman (Fixed Effects vs Random Effects)
from linearmodels.panel import compare
#Membandingkan model Fixed Effects dan Random Effects dengan uji Hausman
comparison = compare({"Fixed Effects": fe_results, "Random Effects": 
re_results})
print(comparison)

                        RandomEffects Estimation Summary                        
Dep. Variable:              Laju_pdrb   R-squared:                        0.3622
Estimator:              RandomEffects   R-squared (Between):             -0.4199
No. Observations:                 152   R-squared (Within):               0.4721
Date:                Thu, Nov 28 2024   R-squared (Overall):              0.0745
Time:                        16:56:36   Log-likelihood                   -355.32
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      16.584
Entities:                          38   P-value                           0.0000
Avg Obs:                       4.0000   Distribution:                   F(5,146)
Min Obs:                       4.0000                                           
Max Obs:                       4.0000   F-statistic (robust):             16.584
                            

## UJI Hetero

In [11]:
from statsmodels.stats.diagnostic import het_breuschpagan

# Mengambil residual dari model Pooled OLS
residuals = fe_results.resids

# Melakukan tes Breusch-Pagan
bp_test = het_breuschpagan(residuals, X)
bp_test_results = {
    "LM Statistic": bp_test[0],
    "LM p-value": bp_test[1],
    "F-statistic": bp_test[2],
    "F p-value": bp_test[3]
}

print("Breusch-Pagan Test Results:", bp_test_results)


Breusch-Pagan Test Results: {'LM Statistic': np.float64(6.6034488192981025), 'LM p-value': np.float64(0.25184146964467474), 'F-statistic': np.float64(1.3261711089959962), 'F p-value': np.float64(0.25624713875769656)}


In [12]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Menghitung VIF untuk setiap variabel independen
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print("Variance Inflation Factors (VIF):")
print(vif_data)


Variance Inflation Factors (VIF):
              Variable        VIF
0                const  13.970264
1                  IPM   7.056189
2                 PDRB   8.176002
3  Realisasi_Investasi   8.016679
4                  tpt   2.304205
5           Gini_Ratio   6.099330
